In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import edesdetectrl.config as general_config
from mlflow.tracking import MlflowClient
import mlflow


client = MlflowClient(general_config.config["mlflow"]["tracking_uri"])
experiment = client.get_experiment_by_name("VanillaBinaryClassificationBaseline")
dict(experiment), experiment.experiment_id


In [ ]:
import os


def get_params(artifact_dir):
    # List of tuples of (step, params_path)
    params_files = [(int(f[7:]), artifact_dir + f) for f in os.listdir(artifact_dir)]
    params_files = [(step, f) for step, f in params_files if step <= 100_000]
    params_files = sorted(params_files, key=lambda x: x[0])
    return params_files


In [ ]:
import numpy as np
from notebooks.plots.results.bce import helpers
from concurrent.futures import ProcessPoolExecutor

pool = ProcessPoolExecutor(5)
all_result = {}
for run in client.search_runs(experiment.experiment_id):
    artifact_dir = run.info.artifact_uri + "/"
    param_files = get_params(artifact_dir)
    latest_params = param_files[-1][1]
    result = pool.submit(helpers.evaluate_with_params, latest_params, "TRAIN")
    all_result[run.info.run_id] = result


In [ ]:
all_result={k:v.result() for k,v in all_result.items()}

In [ ]:
import pickle

with open("notebooks/plots/results/bce/store/exp1_final_TRAIN_params", "wb+") as f:
    pickle.dump(all_result, f)

In [ ]:
all_s = {}
for run_id, r in all_result.items():
    s = {}
    for m in r.values():
        for k, v in m.items():
            s[k] = s.get(k, 0) + v/len(r)
    all_s[run_id] = s

In [ ]:
import matplotlib.pyplot as plt

run_labels = {
    "567a46b69336472cb8e7a3b13688e3f1": "No N-step bootstrapping, no discounting, 100% exploration",
    "410178a1d5ef48f68510a785d045baed": "4-step bootstrapping, no discounting, 100% exploration",
    "0228822522b84e7e8c6aef686b4b4b25": "4-step bootstrapping, discounting, 100% exploration",
    "e9b6431dc438403983c28a1937a02fa8": "4-step bootstrapping, discounting, 20% exploration",
}

fig, ax = plt.subplots(ncols=2, figsize=(12,5))
for i, (run_id, s) in enumerate(all_s.items()):
    ax[0].bar([0 + i / 9], [s["balanced_accuracy"]], width=0.1)
ax[0].set_xticks([], [])
ax[0].set_title("Balanced Accuracy")
ax[0].legend([run_labels[k] for k in all_s.keys()], loc="lower right")
ax[0].set_ylim([0.5, 0.8])

for i, (run_id, s) in enumerate(all_s.items()):
    ax[1].bar([0 + i / 9], [s["gaafd"]], width=0.1)
ax[1].set_xticks([], [])
ax[1].set_title("GaaFD")
ax[1].legend([run_labels[k] for k in all_s.keys()], loc="lower right")
#ax[1].set_ylim([3.2, 3.22])

fig.tight_layout()


In [ ]:
legend_values = []
fig, ax = plt.subplots(ncols=2, figsize=(12, 5))
for run in client.search_runs(experiment.experiment_id):
    loss_metrics = client.get_metric_history(run.info.run_id, "total_loss")
    ax[0].plot(
        [m.step for m in loss_metrics if m.step <= 5_000],
        [m.value for m in loss_metrics if m.step <= 5_000],
    )
    legend_values.append(run_labels[run.info.run_id])
    ax[0].set_ylim([0,0.6])
    ax[0].set_xlabel("Step")
    ax[0].set_ylabel("Loss")
ax[0].legend(legend_values, loc="upper right")

for run in client.search_runs(experiment.experiment_id):
    loss_metrics = client.get_metric_history(run.info.run_id, "total_loss")
    ax[1].plot(
        [m.step for m in loss_metrics if m.step <= 100_000],
        [m.value for m in loss_metrics if m.step <= 100_000],
    )
    ax[1].set_ylim([0,0.6])
    ax[1].set_xlabel("Step")
    ax[1].set_ylabel("Loss")
ax[1].legend(legend_values, loc="upper right")

fig.tight_layout()
